목표 : 파생변수 Derived Variable 만들기
========
- OMI-CF : 영업현금흐름 - 투자현금흐름
- OMS-CF : 영업현금흐름 - 재무현금흐름
- ODI-CF : 영업현금흐름 / 투자현금흐름
- ODS-CF : 영업현금흐름 / 재무현금흐름
- 법인세 변화율
- 순이자비용율(-1) (= 순이자비용 / 총부채(평균?)), 영업이익률(1) (= 영업이익 / 운전자본) 가중조화평균
- 유동비율(-1) (= 유동자산 / 유동부채), 유동부채비율(1) (= 유동부채 / 총자산) 가중조화평균
- 영업레버리지 = 매출총이익 / 영업이익
- 재무레버리지 = 영업이익 / 세전이익

아래의 변수로 파생변수를 만들면 좋겠다
1. CF 대매출액비율
2. 매출액 영업이익률
3. CF 대 차입금 비율
4. 매출액증가율
5. 순운전자본비율
6. 차입금의존도
7. 이윤분배율
8. 자기자본순이익률
9. 부채비율
10. 자기자본구성비율

load modules

In [2]:
import pandas as pd
import numpy as np
import platform

#워닝 메시지
import warnings
warnings.filterwarnings('ignore')

import os
current_directory = os.getcwd()
print("현재 디렉토리:", current_directory)

현재 디렉토리: c:\Users\LG\2ndProject\1_데이터 수집 및 전처리


환경설정

In [3]:
# pd.set_option('display.max_rows', None)
pd.reset_option('display.max_rows')

# pd.set_option('display.max_columns', None)
pd.reset_option('display.max_columns')

pd.set_option('display.max_seq_items', None)

pd.set_option('float_format', '{:.3f}'.format) # 소수점 3개까지 출력

read csv files

In [9]:
unlist = pd.read_csv("../비상장영업이익구성.csv")
onlist = pd.read_csv("../상장영업이익구성.csv")
uncash = pd.read_csv("../data/datasets/외감_현금흐름.csv", index_col=0)
oncash = pd.read_csv("../data/datasets/상장_현금흐름.csv", index_col=0)

# describe

비상장 unlist

In [10]:
unlist

,회사명,거래소코드,회계년도,[U01210000]매출액(영업수익)(*)(천원),[U01220010]매출원가(*)(천원),[U01222170]판매비와관리비(영업비용)(*)(천원),[U01240000]영업외수익(*)(천원),[U01250000]영업외비용(*)(천원),[U01110010] 유동자산(*)(천원),[U01140010] 유동부채(*)(천원),...,[U01B450000000]기타이익(*)(IFRS연결)(천원),[U01B470000000]기타손실(*)(IFRS연결)(천원),[U01B750000000]법인세비용(IFRS연결)(천원),[U01A111038600] 유동자산(*)(IFRS연결)(천원),[U01A811026000] 유동부채(*)(IFRS연결)(천원),[U01B201013300] 이자비용(IFRS연결)(천원),[U01B100012000] 이자수익(IFRS연결)(천원),[U01A100000000]자산(*)(IFRS연결)(천원),[U01B800000000]계속영업이익(손실)(IFRS연결)(천원),[U01A800000000]부채(*)(IFRS연결)(천원)
0,(유)에스케이씨에보닉페록사이드코리아,93338,2019/12,63261273.000,50286525.000,1813279.000,577894.000,61782.000,39035530.000,8482795.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,(유)에스케이씨에보닉페록사이드코리아,93338,2020/12,60407830.000,46715670.000,1714166.000,387346.000,184639.000,37996798.000,8195638.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,(유)에스케이씨에보닉페록사이드코리아,93338,2021/12,68112530.000,53842079.000,1781022.000,376875.000,112899.000,41592405.000,10840688.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,(유)에스케이씨에보닉페록사이드코리아,93338,2022/12,80434278.000,69516849.000,1530379.000,759236.000,230474.000,47773836.000,13131531.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,(유)연세,93779,2019/12,14140804.000,6528909.000,5318473.000,10468.000,106472.000,14754078.000,816481.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243657,주식회사이에스아이,57932,2016/12,791750.000,0.000,628752.000,117104.000,268804.000,305594.000,8215940.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
243658,주식회사이에스아이,57932,2017/12,1058163.000,0.000,1322207.000,909789.000,1772817.000,1355274.000,438327.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
243659,주식회사이에스아이,57932,2018/12,506265.000,0.000,1183949.000,132992.000,354231.000,374093.000,1111500.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
243660,주식회사이에스아이,57932,2019/12,511522.000,0.000,1317739.000,60676.000,195882.000,1554044.000,2567833.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
unlist.describe()

,거래소코드,[U01210000]매출액(영업수익)(*)(천원),[U01220010]매출원가(*)(천원),[U01222170]판매비와관리비(영업비용)(*)(천원),[U01240000]영업외수익(*)(천원),[U01250000]영업외비용(*)(천원),[U01110010] 유동자산(*)(천원),[U01140010] 유동부채(*)(천원),[U01110000]자산(*)(천원),[U01230000]영업이익(손실)(천원),...,[U01B450000000]기타이익(*)(IFRS연결)(천원),[U01B470000000]기타손실(*)(IFRS연결)(천원),[U01B750000000]법인세비용(IFRS연결)(천원),[U01A111038600] 유동자산(*)(IFRS연결)(천원),[U01A811026000] 유동부채(*)(IFRS연결)(천원),[U01B201013300] 이자비용(IFRS연결)(천원),[U01B100012000] 이자수익(IFRS연결)(천원),[U01A100000000]자산(*)(IFRS연결)(천원),[U01B800000000]계속영업이익(손실)(IFRS연결)(천원),[U01A800000000]부채(*)(IFRS연결)(천원)
count,243662.000,243620.000,243620.000,243620.000,243620.000,243620.000,243620.000,243620.000,243620.000,243620.000,...,2739.000,2739.000,2739.000,2739.000,2739.000,2739.000,2739.000,2739.000,2739.000,2739.000
mean,64401.533,20023261.822,15686706.376,3448555.288,842449.906,1097741.741,12873302.322,12030890.457,29427210.861,888000.161,...,1489920.802,1899063.770,626316.850,32377129.881,30432867.698,28481.843,56231.846,70778188.391,-1102741.295,43748043.435
std,24145.478,24791145.772,21542420.235,5713060.493,4185347.780,4021307.222,20933003.503,22190891.739,36457147.456,3983449.661,...,5037449.576,7700803.361,2846196.394,50622888.386,46374342.716,969718.692,1235905.718,94930324.450,17344180.883,69247988.410
min,10037.000,-58226688.000,-52121705.000,-16532421.000,-494580.000,-3689828.000,-48070.000,-953582.000,-40617.000,-137920600.000,...,0.000,-287327.000,-26487029.000,0.000,0.000,0.000,0.000,0.000,-404195336.000,0.000
25%,49054.000,3671549.500,1008633.750,895664.500,48371.000,185424.500,3443558.250,3503297.250,12797107.750,-144854.750,...,77161.000,59701.000,0.000,10061553.500,7537072.500,0.000,0.000,22307055.500,-3427854.000,11962258.000
50%,66573.000,13302925.500,9589942.500,1886888.000,197482.500,415071.500,7973989.500,7204549.000,18811826.000,489163.000,...,322331.000,353838.000,77242.000,20883895.000,17086152.000,0.000,0.000,43941813.000,658925.000,25027342.000
75%,82446.750,26987679.000,21309624.000,3755731.250,593757.500,941620.500,14973438.000,13019747.000,31845025.000,1530631.750,...,1112195.500,1390676.500,783106.500,39615685.500,36443946.000,0.000,0.000,79898290.000,3577534.500,48419248.500
max,107198.000,1124063193.000,1098953666.000,312926130.000,547885676.000,603023333.000,1639437078.000,943002318.000,1640688664.000,248151921.000,...,104826100.000,287931561.000,69086428.000,1051294083.000,877162801.000,42557978.000,54268934.000,1349598765.000,190842139.000,1225169588.000


In [12]:
unlist.columns

Index(['회사명', '거래소코드', '회계년도', '[U01210000]매출액(영업수익)(*)(천원)',
       '[U01220010]매출원가(*)(천원)', '[U01222170]판매비와관리비(영업비용)(*)(천원)',
       '[U01240000]영업외수익(*)(천원)', '[U01250000]영업외비용(*)(천원)',
       '[U01110010]   유동자산(*)(천원)', '[U01140010]   유동부채(*)(천원)',
       '[U01110000]자산(*)(천원)', '[U01230000]영업이익(손실)(천원)',
       '[U01250010]   이자비용(천원)', '[U01210534]   이자수익(천원)',
       '[U01140000]부채(*)(천원)', '[U01B100000000]매출액(수익)(*)(IFRS)(천원)',
       '[U01B200000000]매출원가(*)(IFRS)(천원)',
       '[U01B350000000]판매비와 관리비(물류원가 등 포함)(*)(IFRS)(천원)',
       '[U01B500000000]금융수익(*)(IFRS)(천원)', '[U01B550000000]금융원가(*)(IFRS)(천원)',
       '[U01B450000000]기타이익(*)(IFRS)(천원)', '[U01B470000000]기타손실(*)(IFRS)(천원)',
       '[U01B750000000]법인세비용(IFRS)(천원)', '[U01A111038600]   유동자산(*)(IFRS)(천원)',
       '[U01A811026000]   유동부채(*)(IFRS)(천원)', '[U01A100000000]자산(*)(IFRS)(천원)',
       '[U01B800000000]계속영업이익(손실)(IFRS)(천원)',
       '[U01B201013300]   이자비용(IFRS)(천원)', '[U01B100012000]   이자수익(IFRS)(천원)',
       '[U01A

In [13]:
len(unlist.columns)

45

In [14]:
unlist.isnull().sum().sort_values(ascending=False) / len(unlist) * 100

[U01A800000000]부채(*)(IFRS연결)(천원)                    98.876
[U01B750000000]법인세비용(IFRS연결)(천원)                    98.876
[U01B100000000]매출액(수익)(*)(IFRS연결)(천원)               98.876
[U01B200000000]매출원가(*)(IFRS연결)(천원)                  98.876
[U01B350000000]판매비와 관리비(물류원가 등 포함)(*)(IFRS연결)(천원)   98.876
[U01B500000000]금융수익(*)(IFRS연결)(천원)                  98.876
[U01B450000000]기타이익(*)(IFRS연결)(천원)                  98.876
[U01B470000000]기타손실(*)(IFRS연결)(천원)                  98.876
[U01B550000000]금융원가(*)(IFRS연결)(천원)                  98.876
[U01A111038600]   유동자산(*)(IFRS연결)(천원)               98.876
[U01A811026000]   유동부채(*)(IFRS연결)(천원)               98.876
[U01B201013300]   이자비용(IFRS연결)(천원)                  98.876
[U01B100012000]   이자수익(IFRS연결)(천원)                  98.876
[U01A100000000]자산(*)(IFRS연결)(천원)                    98.876
[U01B800000000]계속영업이익(손실)(IFRS연결)(천원)               98.876
[U01A111038600]   유동자산(*)(IFRS)(천원)                 95.846
[U01A800000000]부채(*)(IFRS)(천원)                      95.8

In [15]:
unlist.isin([0]).sum().sort_values(ascending=False)

[U01210534]   이자수익(천원)                               238877
[U01220010]매출원가(*)(천원)                                46080
[U01250010]   이자비용(천원)                                25964
[U01210000]매출액(영업수익)(*)(천원)                           13806
[U01B201013300]   이자비용(IFRS)(천원)                      10100
[U01B100012000]   이자수익(IFRS)(천원)                       9882
[U01250000]영업외비용(*)(천원)                                4271
[U01B201013300]   이자비용(IFRS연결)(천원)                     2730
[U01B100012000]   이자수익(IFRS연결)(천원)                     2690
[U01B750000000]법인세비용(IFRS)(천원)                         2434
[U01B200000000]매출원가(*)(IFRS)(천원)                       2025
[U01B550000000]금융원가(*)(IFRS)(천원)                        764
[U01240000]영업외수익(*)(천원)                                 662
[U01B470000000]기타손실(*)(IFRS)(천원)                        611
[U01B100000000]매출액(수익)(*)(IFRS)(천원)                     534
[U01B200000000]매출원가(*)(IFRS연결)(천원)                      531
[U01222170]판매비와관리비(영업비용)(*)(천원)         

In [16]:
unlist.isin([np.inf]).sum().sort_values(ascending=False)

회사명                                                  0
[U01A111038600]   유동자산(*)(IFRS)(천원)                  0
[U01A100000000]자산(*)(IFRS)(천원)                       0
[U01B800000000]계속영업이익(손실)(IFRS)(천원)                  0
[U01B201013300]   이자비용(IFRS)(천원)                     0
[U01B100012000]   이자수익(IFRS)(천원)                     0
[U01A800000000]부채(*)(IFRS)(천원)                       0
[U01B100000000]매출액(수익)(*)(IFRS연결)(천원)                0
[U01B200000000]매출원가(*)(IFRS연결)(천원)                   0
[U01B350000000]판매비와 관리비(물류원가 등 포함)(*)(IFRS연결)(천원)    0
[U01B500000000]금융수익(*)(IFRS연결)(천원)                   0
[U01B550000000]금융원가(*)(IFRS연결)(천원)                   0
[U01B450000000]기타이익(*)(IFRS연결)(천원)                   0
[U01B470000000]기타손실(*)(IFRS연결)(천원)                   0
[U01B750000000]법인세비용(IFRS연결)(천원)                     0
[U01A111038600]   유동자산(*)(IFRS연결)(천원)                0
[U01A811026000]   유동부채(*)(IFRS연결)(천원)                0
[U01B201013300]   이자비용(IFRS연결)(천원)                   0
[U01B10001

In [17]:
len(unlist.index)

243662

상장 onlist

In [18]:
onlist

,회사명,거래소코드,회계년도,[U01210000]매출액(영업수익)(*)(천원),[U01220010]매출원가(*)(천원),[U01222170]판매비와관리비(영업비용)(*)(천원),[U01240000]영업외수익(*)(천원),[U01250000]영업외비용(*)(천원),[U01290100](계속사업손익)법인세비용(천원),[U01230000]영업이익(손실)(천원),...,[U01B500000000]금융수익(*)(IFRS연결)(천원),[U01B550000000]금융원가(*)(IFRS연결)(천원),[U01B450000000]기타이익(*)(IFRS연결)(천원),[U01B470000000]기타손실(*)(IFRS연결)(천원),[U01A111038600] 유동자산(*)(IFRS연결)(천원),[U01A811026000] 유동부채(*)(IFRS연결)(천원),[U01A100000000]자산(*)(IFRS연결)(천원),[U01B201013300] 이자비용(IFRS연결)(천원),[U01B100012000] 이자수익(IFRS연결)(천원),[U01A800000000]부채(*)(IFRS연결)(천원)
0,(주)CMG제약,58820,2012/12,16255389.000,11648690.000,8651838.000,181623.000,3029222.000,0.000,-4045140.000,...,131363.000,1289699.000,38548.000,1334858.000,47109213.000,6793095.000,57108035.000,0.000,0.000,25517350.000
1,(주)CMG제약,58820,2013/12,20322140.000,12506817.000,6992776.000,1071508.000,1543788.000,0.000,822547.000,...,946706.000,1266882.000,123559.000,283443.000,38209699.000,5500075.000,55912563.000,0.000,0.000,6915281.000
2,(주)CMG제약,58820,2014/12,22752584.000,14318048.000,7988518.000,481299.000,755814.000,0.000,446018.000,...,330165.000,215794.000,141370.000,540023.000,28300941.000,8719202.000,58068326.000,0.000,0.000,9065682.000
3,(주)CMG제약,58820,2015/12,27041894.000,17237814.000,12374958.000,533965.000,1611756.000,0.000,-2570878.000,...,109662.000,79830.000,424304.000,1532058.000,26271614.000,8849994.000,56532821.000,0.000,0.000,9725933.000
4,(주)CMG제약,58820,2016/12,32969825.000,19663389.000,11575683.000,277853.000,366366.000,-51222.000,1730753.000,...,166851.000,233109.000,111002.000,133257.000,78772201.000,9136755.000,110979867.000,0.000,0.000,9872232.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10916,퓨쳐메디신(주),341170,2018/12,22916.000,47861.000,4201758.000,57114.000,429346.000,0.000,-4226703.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10917,퓨쳐메디신(주),341170,2019/12,0.000,0.000,7162066.000,63694.000,3603639.000,0.000,-7162066.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10918,퓨쳐메디신(주),341170,2020/12,0.000,0.000,5900817.000,111653.000,4713021.000,0.000,-5900817.000,...,35900.000,4692676.000,75753.000,20345.000,7635875.000,24783210.000,9104546.000,0.000,0.000,27089591.000
10919,퓨쳐메디신(주),341170,2021/12,1000000.000,0.000,7499483.000,577023.000,6968919.000,0.000,-6499483.000,...,421255.000,6967502.000,226643.000,1417.000,6710328.000,589671.000,8338518.000,0.000,0.000,1160419.000


In [19]:
onlist.describe()

,거래소코드,[U01210000]매출액(영업수익)(*)(천원),[U01220010]매출원가(*)(천원),[U01222170]판매비와관리비(영업비용)(*)(천원),[U01240000]영업외수익(*)(천원),[U01250000]영업외비용(*)(천원),[U01290100](계속사업손익)법인세비용(천원),[U01230000]영업이익(손실)(천원),[U01110010] 유동자산(*)(천원),[U01140010] 유동부채(*)(천원),...,[U01B500000000]금융수익(*)(IFRS연결)(천원),[U01B550000000]금융원가(*)(IFRS연결)(천원),[U01B450000000]기타이익(*)(IFRS연결)(천원),[U01B470000000]기타손실(*)(IFRS연결)(천원),[U01A111038600] 유동자산(*)(IFRS연결)(천원),[U01A811026000] 유동부채(*)(IFRS연결)(천원),[U01A100000000]자산(*)(IFRS연결)(천원),[U01B201013300] 이자비용(IFRS연결)(천원),[U01B100012000] 이자수익(IFRS연결)(천원),[U01A800000000]부채(*)(IFRS연결)(천원)
count,10921.000,10622.000,10622.000,10622.000,10622.000,10622.000,10622.000,10622.000,10622.000,10622.000,...,6287.000,6287.000,6287.000,6287.000,6287.000,6287.000,6287.000,6287.000,6287.000,6287.000
mean,154080.032,41917055.551,30543197.054,9768951.760,2962276.097,4737650.020,554211.891,1604906.748,35329887.969,18895745.016,...,2078101.653,3314153.667,2039070.969,2925478.874,55990944.531,31662754.652,111689098.486,3583.527,18379.894,45348895.498
std,132806.978,57978025.627,39171285.010,11777002.209,8537726.793,13305557.962,5708553.718,21750313.322,48453265.593,23503169.858,...,5579096.982,12600760.675,6609305.030,6786340.765,71935857.151,40758891.894,138870280.645,271601.899,302966.843,83471704.900
min,40.000,0.000,0.000,-3869865.000,0.000,-5159721.000,-54697041.000,-74431218.000,37121.000,0.000,...,0.000,-11582751.000,0.000,-554429.000,0.000,0.000,0.000,0.000,0.000,0.000
25%,54090.000,15018268.000,8421664.250,3797741.750,420410.000,573451.500,0.000,-1785620.250,13630528.500,5155072.750,...,251078.000,429575.500,197081.000,242697.500,25247810.500,9502377.500,50386110.500,0.000,0.000,13884695.000
50%,114840.000,30948137.000,21422844.000,6413062.000,1080093.500,1630159.500,32783.500,1001600.500,26189770.500,11908207.000,...,687220.000,1238572.000,671879.000,965550.000,41127047.000,20906626.000,81851062.000,0.000,0.000,28830619.000
75%,229000.000,55781408.250,41293993.250,11249655.000,2735542.750,4606468.750,659661.000,3696716.750,45226305.500,24607003.500,...,1825556.000,3161956.000,1808732.000,3038693.000,66037730.000,39865464.500,128920463.500,0.000,0.000,52324388.000
max,950220.000,2847242798.000,1482369847.000,317774408.000,343405500.000,798922538.000,375268635.000,1362655330.000,2486930963.000,729039798.000,...,229630031.000,754588577.000,306034715.000,236578081.000,2577980352.000,779869185.000,3261922382.000,21515379.000,10138689.000,2092305299.000


In [20]:
onlist.isnull().sum().sort_values(ascending=False) /len(onlist) * 100
# 결측치 비율

[U01A800000000]부채(*)(IFRS연결)(천원)                    42.432
[U01B470000000]기타손실(*)(IFRS연결)(천원)                  42.432
[U01B100000000]매출액(수익)(*)(IFRS연결)(천원)               42.432
[U01B200000000]매출원가(*)(IFRS연결)(천원)                  42.432
[U01B350000000]판매비와 관리비(물류원가 등 포함)(*)(IFRS연결)(천원)   42.432
[U01B500000000]금융수익(*)(IFRS연결)(천원)                  42.432
[U01B450000000]기타이익(*)(IFRS연결)(천원)                  42.432
[U01B550000000]금융원가(*)(IFRS연결)(천원)                  42.432
[U01A111038600]   유동자산(*)(IFRS연결)(천원)               42.432
[U01A811026000]   유동부채(*)(IFRS연결)(천원)               42.432
[U01A100000000]자산(*)(IFRS연결)(천원)                    42.432
[U01B201013300]   이자비용(IFRS연결)(천원)                  42.432
[U01B100012000]   이자수익(IFRS연결)(천원)                  42.432
[U01250000]영업외비용(*)(연결)(천원)                         42.340
[U01240010]   이자수익(연결)(천원)                          42.340
[U01221302]   이자비용(연결)(천원)                          42.340
[U01110000]자산(*)(연결)(천원)                            42.3

In [21]:
onlist.isin([0]).sum().sort_values(ascending=False)

[U01221302]   이자비용(천원)                               10619
[U01B201013300]   이자비용(IFRS)(천원)                      9162
[U01B100012000]   이자수익(IFRS)(천원)                      9141
[U01221302]   이자비용(연결)(천원)                            6291
[U01B201013300]   이자비용(IFRS연결)(천원)                    6282
[U01B100012000]   이자수익(IFRS연결)(천원)                    6242
[U01290100](계속사업손익)법인세비용(천원)                          2177
[U01220010]매출원가(*)(천원)                                 720
[U01B200000000]매출원가(*)(IFRS)(천원)                       620
[U01220010]매출원가(*)(연결)(천원)                             410
[U01B200000000]매출원가(*)(IFRS연결)(천원)                     407
[U01B550000000]금융원가(*)(IFRS)(천원)                       239
[U01B550000000]금융원가(*)(IFRS연결)(천원)                      89
[U01210000]매출액(영업수익)(*)(천원)                             81
[U01B100000000]매출액(수익)(*)(IFRS)(천원)                     51
[U01B470000000]기타손실(*)(IFRS)(천원)                        50
[U01B470000000]기타손실(*)(IFRS연결)(천원)                      

In [22]:
onlist.isin([np.inf]).sum().sort_values(ascending=False)

회사명                                                  0
[U01B100000000]매출액(수익)(*)(IFRS)(천원)                  0
[U01B350000000]판매비와 관리비(물류원가 등 포함)(*)(IFRS)(천원)      0
[U01B500000000]금융수익(*)(IFRS)(천원)                     0
[U01B550000000]금융원가(*)(IFRS)(천원)                     0
[U01B450000000]기타이익(*)(IFRS)(천원)                     0
[U01B470000000]기타손실(*)(IFRS)(천원)                     0
[U01A111038600]   유동자산(*)(IFRS)(천원)                  0
[U01A811026000]   유동부채(*)(IFRS)(천원)                  0
[U01A100000000]자산(*)(IFRS)(천원)                       0
[U01B201013300]   이자비용(IFRS)(천원)                     0
[U01B100012000]   이자수익(IFRS)(천원)                     0
[U01A800000000]부채(*)(IFRS)(천원)                       0
[U01B100000000]매출액(수익)(*)(IFRS연결)(천원)                0
[U01B200000000]매출원가(*)(IFRS연결)(천원)                   0
[U01B350000000]판매비와 관리비(물류원가 등 포함)(*)(IFRS연결)(천원)    0
[U01B500000000]금융수익(*)(IFRS연결)(천원)                   0
[U01B550000000]금융원가(*)(IFRS연결)(천원)                   0
[U01B45000

전처리?

In [23]:
unlist.replace(0, np.nan, inplace=True)
onlist.replace(0, np.nan, inplace=True)

# 상장 onlist

In [24]:
onlist

,회사명,거래소코드,회계년도,[U01210000]매출액(영업수익)(*)(천원),[U01220010]매출원가(*)(천원),[U01222170]판매비와관리비(영업비용)(*)(천원),[U01240000]영업외수익(*)(천원),[U01250000]영업외비용(*)(천원),[U01290100](계속사업손익)법인세비용(천원),[U01230000]영업이익(손실)(천원),...,[U01B500000000]금융수익(*)(IFRS연결)(천원),[U01B550000000]금융원가(*)(IFRS연결)(천원),[U01B450000000]기타이익(*)(IFRS연결)(천원),[U01B470000000]기타손실(*)(IFRS연결)(천원),[U01A111038600] 유동자산(*)(IFRS연결)(천원),[U01A811026000] 유동부채(*)(IFRS연결)(천원),[U01A100000000]자산(*)(IFRS연결)(천원),[U01B201013300] 이자비용(IFRS연결)(천원),[U01B100012000] 이자수익(IFRS연결)(천원),[U01A800000000]부채(*)(IFRS연결)(천원)
0,(주)CMG제약,58820,2012/12,16255389.000,11648690.000,8651838.000,181623.000,3029222.000,NaN,-4045140.000,...,131363.000,1289699.000,38548.000,1334858.000,47109213.000,6793095.000,57108035.000,NaN,NaN,25517350.000
1,(주)CMG제약,58820,2013/12,20322140.000,12506817.000,6992776.000,1071508.000,1543788.000,NaN,822547.000,...,946706.000,1266882.000,123559.000,283443.000,38209699.000,5500075.000,55912563.000,NaN,NaN,6915281.000
2,(주)CMG제약,58820,2014/12,22752584.000,14318048.000,7988518.000,481299.000,755814.000,NaN,446018.000,...,330165.000,215794.000,141370.000,540023.000,28300941.000,8719202.000,58068326.000,NaN,NaN,9065682.000
3,(주)CMG제약,58820,2015/12,27041894.000,17237814.000,12374958.000,533965.000,1611756.000,NaN,-2570878.000,...,109662.000,79830.000,424304.000,1532058.000,26271614.000,8849994.000,56532821.000,NaN,NaN,9725933.000
4,(주)CMG제약,58820,2016/12,32969825.000,19663389.000,11575683.000,277853.000,366366.000,-51222.000,1730753.000,...,166851.000,233109.000,111002.000,133257.000,78772201.000,9136755.000,110979867.000,NaN,NaN,9872232.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10916,퓨쳐메디신(주),341170,2018/12,22916.000,47861.000,4201758.000,57114.000,429346.000,NaN,-4226703.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10917,퓨쳐메디신(주),341170,2019/12,NaN,NaN,7162066.000,63694.000,3603639.000,NaN,-7162066.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10918,퓨쳐메디신(주),341170,2020/12,NaN,NaN,5900817.000,111653.000,4713021.000,NaN,-5900817.000,...,35900.000,4692676.000,75753.000,20345.000,7635875.000,24783210.000,9104546.000,NaN,NaN,27089591.000
10919,퓨쳐메디신(주),341170,2021/12,1000000.000,NaN,7499483.000,577023.000,6968919.000,NaN,-6499483.000,...,421255.000,6967502.000,226643.000,1417.000,6710328.000,589671.000,8338518.000,NaN,NaN,1160419.000


상장매출액

In [25]:
onlist[
    [
        '[U01B100000000]매출액(수익)(*)(IFRS연결)(천원)',
        '[U01B100000000]매출액(수익)(*)(IFRS)(천원)',
        '[U01210000]매출액(영업수익)(*)(연결)(천원)',
        '[U01210000]매출액(영업수익)(*)(천원)'
    ]
].isna().sum()

[U01B100000000]매출액(수익)(*)(IFRS연결)(천원)    4652
[U01B100000000]매출액(수익)(*)(IFRS)(천원)      1807
[U01210000]매출액(영업수익)(*)(연결)(천원)          4646
[U01210000]매출액(영업수익)(*)(천원)               380
dtype: int64

In [26]:
상장매출액 = onlist['[U01B100000000]매출액(수익)(*)(IFRS연결)(천원)'].fillna(onlist['[U01B100000000]매출액(수익)(*)(IFRS)(천원)'])
상장매출액 = 상장매출액.fillna(onlist['[U01210000]매출액(영업수익)(*)(연결)(천원)'])
상장매출액 = 상장매출액.fillna(onlist['[U01210000]매출액(영업수익)(*)(천원)'])
onlist['상장매출액']  = 상장매출액
onlist.drop(
    [
        '[U01B100000000]매출액(수익)(*)(IFRS연결)(천원)',
        '[U01B100000000]매출액(수익)(*)(IFRS)(천원)',
        '[U01210000]매출액(영업수익)(*)(연결)(천원)',
        '[U01210000]매출액(영업수익)(*)(천원)'    
    ],
    axis=1,
    inplace=True
)
onlist['상장매출액'].isnull().value_counts()

상장매출액
False    10626
True       295
Name: count, dtype: int64

In [27]:
onlist['상장매출액'].describe()

count        10626.000
mean      54019191.999
std       77432361.498
min           1975.000
25%       17958237.750
50%       37420449.500
75%       66709986.500
max     2932001440.000
Name: 상장매출액, dtype: float64

상장매출원가

In [28]:
상장매출원가 = onlist['[U01B200000000]매출원가(*)(IFRS연결)(천원)'].fillna(onlist['[U01B200000000]매출원가(*)(IFRS)(천원)'])
상장매출원가 = 상장매출원가.fillna(onlist['[U01220010]매출원가(*)(연결)(천원)'])
상장매출원가 = 상장매출원가.fillna(onlist['[U01220010]매출원가(*)(천원)'])
onlist['상장매출원가'] = 상장매출원가
print(
    onlist[
        [
            '[U01B200000000]매출원가(*)(IFRS연결)(천원)',
            '[U01B200000000]매출원가(*)(IFRS)(천원)',
            '[U01220010]매출원가(*)(연결)(천원)',
            '[U01220010]매출원가(*)(천원)'
        ]
    ].isna().sum()
)
onlist.drop(
    [
        '[U01B200000000]매출원가(*)(IFRS연결)(천원)',
        '[U01B200000000]매출원가(*)(IFRS)(천원)',
        '[U01220010]매출원가(*)(연결)(천원)',
        '[U01220010]매출원가(*)(천원)'
    ],
    axis=1,
    inplace=True
)
print(상장매출원가.isna().value_counts())

[U01B200000000]매출원가(*)(IFRS연결)(천원)    5041
[U01B200000000]매출원가(*)(IFRS)(천원)      2376
[U01220010]매출원가(*)(연결)(천원)            5034
[U01220010]매출원가(*)(천원)                1019
dtype: int64
[U01B200000000]매출원가(*)(IFRS연결)(천원)
False    10025
True       896
Name: count, dtype: int64


In [29]:
onlist['상장매출원가'].describe()

count        10025.000
mean      41913878.161
std       58368922.574
min           1697.000
25%       12785809.000
50%       27742307.000
75%       51559111.000
max     1539006778.000
Name: 상장매출원가, dtype: float64

상장매출총이익

In [30]:
onlist['상장매출총이익'] = onlist['상장매출액'] - onlist['상장매출원가']

상장판관비

In [31]:
상장판관비 = onlist['[U01B350000000]판매비와 관리비(물류원가 등 포함)(*)(IFRS연결)(천원)'].fillna(onlist['[U01B350000000]판매비와 관리비(물류원가 등 포함)(*)(IFRS)(천원)'])
상장판관비 = 상장판관비.fillna(onlist['[U01222170]판매비와관리비(영업비용)(*)(연결)(천원)'])
상장판관비 = 상장판관비.fillna(onlist['[U01222170]판매비와관리비(영업비용)(*)(천원)'])
onlist['상장판관비'] = 상장판관비
print(
    onlist[
        [
            '[U01B350000000]판매비와 관리비(물류원가 등 포함)(*)(IFRS연결)(천원)',
            '[U01B350000000]판매비와 관리비(물류원가 등 포함)(*)(IFRS)(천원)',
            '[U01222170]판매비와관리비(영업비용)(*)(연결)(천원)',
            '[U01222170]판매비와관리비(영업비용)(*)(천원)'
        ]
    ].isna().sum()
)
onlist.drop(
    [
        '[U01B350000000]판매비와 관리비(물류원가 등 포함)(*)(IFRS연결)(천원)',
        '[U01B350000000]판매비와 관리비(물류원가 등 포함)(*)(IFRS)(천원)',
        '[U01222170]판매비와관리비(영업비용)(*)(연결)(천원)',
        '[U01222170]판매비와관리비(영업비용)(*)(천원)'
    ],
    axis=1,
    inplace=True
)
상장판관비.isna().value_counts()

[U01B350000000]판매비와 관리비(물류원가 등 포함)(*)(IFRS연결)(천원)    4635
[U01B350000000]판매비와 관리비(물류원가 등 포함)(*)(IFRS)(천원)      1759
[U01222170]판매비와관리비(영업비용)(*)(연결)(천원)                  4629
[U01222170]판매비와관리비(영업비용)(*)(천원)                       303
dtype: int64


[U01B350000000]판매비와 관리비(물류원가 등 포함)(*)(IFRS연결)(천원)
False    10697
True       224
Name: count, dtype: int64

In [32]:
onlist['상장판관비'].describe()

count       10697.000
mean     12615269.282
std      16319690.276
min      -2762804.000
25%       4466974.000
50%       7981626.000
75%      14491411.000
max     312674788.000
Name: 상장판관비, dtype: float64

상장영업이익

In [33]:
onlist.columns

Index(['회사명', '거래소코드', '회계년도', '[U01240000]영업외수익(*)(천원)',
       '[U01250000]영업외비용(*)(천원)', '[U01290100](계속사업손익)법인세비용(천원)',
       '[U01230000]영업이익(손실)(천원)', '[U01110010]   유동자산(*)(천원)',
       '[U01140010]   유동부채(*)(천원)', '[U01110000]자산(*)(천원)',
       '[U01221302]   이자비용(천원)', '[U01240010]   이자수익(천원)',
       '[U01140000]부채(*)(천원)', '[U01240000]영업외수익(*)(연결)(천원)',
       '[U01250000]영업외비용(*)(연결)(천원)', '[U01110010]   유동자산(*)(연결)(천원)',
       '[U01140010]   유동부채(*)(연결)(천원)', '[U01110000]자산(*)(연결)(천원)',
       '[U01221302]   이자비용(연결)(천원)', '[U01240010]   이자수익(연결)(천원)',
       '[U01140000]부채(*)(연결)(천원)', '[U01B500000000]금융수익(*)(IFRS)(천원)',
       '[U01B550000000]금융원가(*)(IFRS)(천원)', '[U01B450000000]기타이익(*)(IFRS)(천원)',
       '[U01B470000000]기타손실(*)(IFRS)(천원)',
       '[U01A111038600]   유동자산(*)(IFRS)(천원)',
       '[U01A811026000]   유동부채(*)(IFRS)(천원)', '[U01A100000000]자산(*)(IFRS)(천원)',
       '[U01B201013300]   이자비용(IFRS)(천원)', '[U01B100012000]   이자수익(IFRS)(천원)',
       '[U01A800000000]부채(*)

In [34]:
onlist['상장영업이익'] = onlist['상장매출총이익'] - onlist['상장판관비']

In [35]:
onlist['상장영업이익'].fillna(onlist['[U01230000]영업이익(손실)(천원)'], inplace=True)

In [36]:
onlist.drop(
    labels = [
        '[U01230000]영업이익(손실)(천원)'
    ],
    axis=1,
    inplace=True
)

상장영업외비용

In [46]:
onlist['[U01B450000000]기타이익(*)(IFRS연결)(천원)'].isna().sum()
# / len(onlist['[U01B450000000]기타이익(*)(IFRS연결)(천원)'])

4673

In [43]:
onlist['[U01B500000000]금융수익(*)(IFRS연결)(천원)'].isna().sum() / len(onlist['[U01B500000000]금융수익(*)(IFRS연결)(천원)'])

0.4263345847449867

In [55]:
상장국제연결영업외비용 = onlist['[U01B550000000]금융원가(*)(IFRS연결)(천원)'].fillna(0)\
    + onlist['[U01B470000000]기타손실(*)(IFRS연결)(천원)'].fillna(0)\
        - onlist['[U01B500000000]금융수익(*)(IFRS연결)(천원)'].fillna(0)\
            - onlist['[U01B450000000]기타이익(*)(IFRS연결)(천원)'].fillna(0)
상장국제연결영업외비용.replace(0, np.nan, inplace=True)

In [56]:
상장국제개별영업외비용 = onlist['[U01B550000000]금융원가(*)(IFRS)(천원)'].fillna(0)\
    + onlist['[U01B470000000]기타손실(*)(IFRS)(천원)'].fillna(0)\
        - onlist['[U01B500000000]금융수익(*)(IFRS)(천원)'].fillna(0)\
            - onlist['[U01B450000000]기타이익(*)(IFRS)(천원)'].fillna(0)
상장국제개별영업외비용.replace(0, np.nan, inplace=True)

In [57]:
상장일반연결영업외비용 = onlist['[U01250000]영업외비용(*)(연결)(천원)'].fillna(0) \
    - onlist['[U01240000]영업외수익(*)(연결)(천원)'].fillna(0)
상장일반연결영업외비용.replace(0, np.nan, inplace=True)

In [60]:
상장일반개별영업외비용 = onlist['[U01250000]영업외비용(*)(천원)'].fillna(0) \
        - onlist['[U01240000]영업외수익(*)(천원)'].fillna(0)
상장일반개별영업외비용.replace(0, np.nan, inplace=True)

In [62]:
상장영업외비용 = 상장국제연결영업외비용.fillna(상장국제개별영업외비용)
상장영업외비용 = 상장영업외비용.fillna(상장일반연결영업외비용)
상장영업외비용 = 상장영업외비용.fillna(상장일반개별영업외비용)
onlist['상장영업외비용'] = 상장영업외비용
onlist.drop(
    [
        '[U01B550000000]금융원가(*)(IFRS연결)(천원)',
        '[U01B470000000]기타손실(*)(IFRS연결)(천원)',
        '[U01B500000000]금융수익(*)(IFRS연결)(천원)',
        '[U01B450000000]기타이익(*)(IFRS연결)(천원)',
        '[U01B550000000]금융원가(*)(IFRS)(천원)',
        '[U01B470000000]기타손실(*)(IFRS)(천원)',
        '[U01B500000000]금융수익(*)(IFRS)(천원)',
        '[U01B450000000]기타이익(*)(IFRS)(천원)',
        '[U01250000]영업외비용(*)(연결)(천원)',
        '[U01240000]영업외수익(*)(연결)(천원)',
        '[U01250000]영업외비용(*)(천원)',
        '[U01240000]영업외수익(*)(천원)'
    ],
    axis=1,
    inplace=True
)
상장영업외비용.isna().value_counts()

False    10697
True       224
Name: count, dtype: int64

상장세전이익

In [63]:
상장세전이익 = onlist['상장영업이익'] - onlist['상장영업외비용']

In [18]:
영업활동현금흐름 = KOSPI_KOSDAQ['[U01D100000000]영업활동으로 인한 현금흐름(간접법)(*)(IFRS연결)(천원)']
영업활동현금흐름 = 영업활동현금흐름.fillna(KOSPI_KOSDAQ['[U01D100000000]영업활동으로 인한 현금흐름(간접법)(*)(IFRS)(천원)'])
영업활동현금흐름 = 영업활동현금흐름.fillna(KOSPI_KOSDAQ['[U01410000]영업활동으로 인한 현금흐름(*)(연결)(천원)'])
영업활동현금흐름 = 영업활동현금흐름.fillna(KOSPI_KOSDAQ['[U01410000]영업활동으로 인한 현금흐름(*)(천원)'])
print(영업활동현금흐름.isna().value_counts())
KOSPI_KOSDAQ['영업활동으로 인한 현금흐름'] = 영업활동현금흐름
KOSPI_KOSDAQ.drop(labels=['[U01D100000000]영업활동으로 인한 현금흐름(간접법)(*)(IFRS연결)(천원)', '[U01D100000000]영업활동으로 인한 현금흐름(간접법)(*)(IFRS)(천원)', '[U01410000]영업활동으로 인한 현금흐름(*)(연결)(천원)', '[U01410000]영업활동으로 인한 현금흐름(*)(천원)'], axis=1, inplace=True)

[U01D100000000]영업활동으로 인한 현금흐름(간접법)(*)(IFRS연결)(천원)
False    7895
True      199
Name: count, dtype: int64


# KONEX 시장지표 만들기

In [44]:
df2 = pd.read_csv("./datasets/TS2000/상장/KONEX_시장지표자료.csv", encoding='cp949')
df3 = pd.read_csv("./datasets/TS2000/상장/KONEX_주당.csv", encoding='cp949')
df4 = pd.read_csv("./datasets/TS2000/상장/KONEX_종가.csv", encoding='cp949')

In [45]:
df2.columns

Index(['회사명', '거래소코드', '회계년도', '종가(원)', '현재까지 발행한 주식의 총수(주)',
       '현재까지 감소한 주식의 총수(주)', '발행주식의 총수 (현재 발행한 주식수 - 현재 감소한 주식수)(주)',
       '발행주식수(주)', '발행주식수(주).1', '[U01B840000000]당기순이익(손실)(요약)(IFRS연결)(백만원)',
       '[U01C500011300]   당기순이익(총포괄손익)(요약)(IFRS연결)(백만원)',
       '[U01D101000000]   당기순손익(요약)(IFRS연결)(백만원)',
       '[U01E100099100]   당기순이익(순손실)(요약)(IFRS연결)(백만원)',
       '[U01B840000000]당기순이익(손실)(요약)(IFRS)(백만원)',
       '[U01C500011300]   당기순이익(총포괄손익)(요약)(IFRS)(백만원)',
       '[U01D101000000]   당기순손익(요약)(IFRS)(백만원)',
       '[U01E100099100]   당기순이익(순손실)(요약)(IFRS)(백만원)',
       '[U01119300]      *당기순이익(요약)(백만원)', '[U01218800]당기순이익(요약)(백만원)',
       '[U01219300]CPA수정후 당기순이익(요약)(백만원)', '[U01311400]   당기순이익(요약)(백만원)',
       '[U01410100]   당기순이익(요약)(백만원)', '[U01550700]   당기순이익(요약)(백만원)',
       '[U01A600000000]자본(*)(요약)(IFRS연결)(백만원)',
       '[U01A600000000]자본(*)(IFRS연결)(백만원)',
       '[U01A600000000]자본(*)(요약)(IFRS)(백만원)', '[U01A600000000]자본(*)(IFRS)(천원)',
       '[U01119400]자본총계(요약

In [46]:
result_df1 = pd.merge(df2, df3, on=['회사명', '거래소코드', '회계년도'], how='inner')
result_df2 = pd.merge(result_df1, df4, on=['거래소코드', '회계년도'], how='inner')

In [47]:
result_df2.to_csv("./datasets/TS2000/상장/KONEX_시장지표.csv")

In [48]:
df4

,회사명,거래소코드,회계년도,종가(원)
0,SGA시스템즈,232290,2016/03,3240
1,SGA시스템즈,232290,2016/04,5000
2,SGA시스템즈,232290,2016/05,4350
3,SGA시스템즈,232290,2016/06,4150
4,SGA시스템즈,232290,2016/07,4300
...,...,...,...,...
14482,휴벡셀,212310,2023/06,2390
14483,휴벡셀,212310,2023/07,3335
14484,휴벡셀,212310,2023/08,3075
14485,휴벡셀,212310,2023/09,3700


In [49]:
KONEX = result_df2.copy()

In [50]:
KONEX.shape

(1477, 56)

In [51]:
KONEX

,회사명_x,거래소코드,회계년도,종가(원)_x,현재까지 발행한 주식의 총수(주),현재까지 감소한 주식의 총수(주),발행주식의 총수 (현재 발행한 주식수 - 현재 감소한 주식수)(주),발행주식수(주),발행주식수(주).1,[U01B840000000]당기순이익(손실)(요약)(IFRS연결)(백만원),...,[U01219000]주당순이익(요약)(원),[U01B951020000]*주당순손익(IFRS연결)(원),[U01B951020000]*주당순손익(IFRS)(원),[U01290400]*주당순손익(연결)(원),[U01290400]*주당순손익(원),당기순이익(천원),주당순이익(원),주당순자산(원),회사명_y,종가(원)_y
0,(주)굿센,243870,2016/12,NaN,1345951.0,0.0,1345951.0,1345951.0,1345951.0,NaN,...,599.0,NaN,599.0,NaN,599.0,1167810.0,599.0,0.0,굿센,14700
1,(주)굿센,243870,2016/12,NaN,1345951.0,0.0,1345951.0,1345951.0,1345951.0,NaN,...,599.0,NaN,599.0,NaN,599.0,1167810.0,599.0,0.0,굿센,14700
2,(주)굿센,243870,2016/12,NaN,0.0,0.0,0.0,1345951.0,1345951.0,NaN,...,599.0,NaN,599.0,NaN,599.0,1167810.0,599.0,0.0,굿센,14700
3,(주)굿센,243870,2017/12,NaN,1345951.0,0.0,1345951.0,1345951.0,1345951.0,NaN,...,10.0,NaN,10.0,NaN,10.0,12709.0,10.0,0.0,굿센,50500
4,(주)굿센,243870,2017/12,NaN,0.0,0.0,0.0,1345951.0,1345951.0,NaN,...,10.0,NaN,10.0,NaN,10.0,12709.0,10.0,0.0,굿센,50500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1472,태양기계(주),116100,2021/12,NaN,6239986.0,0.0,6239986.0,6239986.0,6239986.0,NaN,...,155.0,NaN,NaN,NaN,155.0,966411.0,155.0,0.0,태양기계 주식회사,799
1473,태양기계(주),116100,2022/12,NaN,6239986.0,0.0,6239986.0,6239986.0,6239986.0,NaN,...,128.0,NaN,NaN,NaN,128.0,798579.0,128.0,0.0,태양기계 주식회사,1165
1474,태양기계(주),116100,2022/12,NaN,6239986.0,0.0,6239986.0,6239986.0,6239986.0,NaN,...,128.0,NaN,NaN,NaN,128.0,798579.0,128.0,0.0,태양기계 주식회사,1165
1475,퓨쳐메디신(주),341170,2022/12,NaN,9275470.0,0.0,9275470.0,9275470.0,9275470.0,-7278.0,...,-755.0,-785.0,-755.0,-785.0,-755.0,-6999528.0,-755.0,0.0,퓨쳐메디신,5090


In [52]:
KONEX.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1477 entries, 0 to 1476
Data columns (total 56 columns):
 #   Column                                                  Non-Null Count  Dtype  
---  ------                                                  --------------  -----  
 0   회사명_x                                                   1477 non-null   object 
 1   거래소코드                                                   1477 non-null   int64  
 2   회계년도                                                    1477 non-null   object 
 3   종가(원)_x                                                 0 non-null      float64
 4   현재까지 발행한 주식의 총수(주)                                      1477 non-null   float64
 5   현재까지 감소한 주식의 총수(주)                                      1477 non-null   float64
 6   발행주식의 총수 (현재 발행한 주식수 - 현재 감소한 주식수)(주)                   1477 non-null   float64
 7   발행주식수(주)                                                1477 non-null   float64
 8   발행주식수(주).1                            

In [53]:
KONEX.describe()

,거래소코드,종가(원)_x,현재까지 발행한 주식의 총수(주),현재까지 감소한 주식의 총수(주),발행주식의 총수 (현재 발행한 주식수 - 현재 감소한 주식수)(주),발행주식수(주),발행주식수(주).1,[U01B840000000]당기순이익(손실)(요약)(IFRS연결)(백만원),[U01C500011300] 당기순이익(총포괄손익)(요약)(IFRS연결)(백만원),[U01D101000000] 당기순손익(요약)(IFRS연결)(백만원),...,[U01B951020000]*주당순손익(요약)(IFRS)(원),[U01219000]주당순이익(요약)(원),[U01B951020000]*주당순손익(IFRS연결)(원),[U01B951020000]*주당순손익(IFRS)(원),[U01290400]*주당순손익(연결)(원),[U01290400]*주당순손익(원),당기순이익(천원),주당순이익(원),주당순자산(원),종가(원)_y
count,1477.000000,0.0,1.477000e+03,1.477000e+03,1.477000e+03,1.477000e+03,1.477000e+03,337.000000,337.000000,337.000000,...,733.000000,1477.000000,337.000000,733.000000,395.000000,1477.000000,1.477000e+03,1477.000000,1477.0,1477.000000
mean,219050.264049,NaN,6.028314e+06,1.298351e+06,4.729963e+06,6.267489e+06,6.267489e+06,-1430.798220,-1367.445104,-1487.798220,...,-291.190996,-638.721733,-44.685460,-296.458390,-461.893671,-641.926879,-1.483745e+06,-603.338524,0.0,8050.086662
std,74508.018080,NaN,7.897790e+06,4.845530e+06,5.511475e+06,5.599319e+06,5.599319e+06,4783.729765,4757.255903,4660.128992,...,1152.201558,4367.617447,727.699528,1152.334009,4039.075984,4367.353507,4.442582e+06,4329.845520,0.0,8782.465477
min,44990.000000,NaN,0.000000e+00,0.000000e+00,0.000000e+00,4.627760e+05,4.627760e+05,-20264.000000,-20152.000000,-20264.000000,...,-6110.000000,-91409.000000,-2653.000000,-6110.000000,-45779.000000,-91409.000000,-4.086658e+07,-91409.000000,0.0,48.000000
25%,176750.000000,NaN,1.517092e+06,0.000000e+00,1.262500e+06,3.075000e+06,3.075000e+06,-3537.000000,-3251.000000,-3397.000000,...,-582.000000,-709.000000,-146.000000,-632.000000,-392.000000,-709.000000,-2.784710e+06,-680.000000,0.0,2520.000000
50%,223220.000000,NaN,3.636360e+06,0.000000e+00,3.449586e+06,4.774581e+06,4.774581e+06,-980.000000,-930.000000,-543.000000,...,-103.000000,-146.000000,0.000000,-105.000000,0.000000,-156.000000,-7.022000e+05,-144.000000,0.0,4990.000000
75%,258540.000000,NaN,7.291098e+06,4.016000e+05,6.239986e+06,7.511064e+06,7.511064e+06,857.000000,858.000000,477.000000,...,170.000000,155.000000,0.000000,170.000000,77.000000,155.000000,5.760360e+05,161.000000,0.0,10650.000000
max,448780.000000,NaN,7.350480e+07,4.762597e+07,4.442780e+07,4.442780e+07,4.442780e+07,19342.000000,19326.000000,19342.000000,...,3433.000000,3433.000000,3956.000000,3433.000000,3956.000000,3433.000000,2.635176e+07,3433.000000,0.0,79900.000000


In [54]:
KONEX.isna().sum().sort_values(ascending=False)

종가(원)_x                                                   1477
[U01B951020000]*주당순손익(IFRS연결)(원)                          1140
[U01B100000000]매출액(수익)(요약)(IFRS연결)(백만원)                   1140
[U01D100000000]영업활동으로 인한 현금흐름(간접법)(*)(IFRS연결)(천원)         1140
[U01D100000000]영업활동으로 인한 현금흐름(간접법)(*)(요약)(IFRS연결)(백만원)    1140
[U01B100000000]매출액(수익)(*)(IFRS연결)(천원)                     1140
[U01A600000000]자본(*)(IFRS연결)(백만원)                         1140
[U01B840000000]당기순이익(손실)(요약)(IFRS연결)(백만원)                 1140
[U01C500011300]   당기순이익(총포괄손익)(요약)(IFRS연결)(백만원)           1140
[U01D101000000]   당기순손익(요약)(IFRS연결)(백만원)                  1140
[U01E100099100]   당기순이익(순손실)(요약)(IFRS연결)(백만원)             1140
[U01A600000000]자본(*)(요약)(IFRS연결)(백만원)                     1140
[U01B951020000]*주당순손익(요약)(IFRS연결)(원)                      1140
[U01410000]영업활동으로 인한 현금흐름(*)(연결)(천원)                      1082
[U01210000]매출액(영업수익)(*)(연결)(천원)                           1082
[U01160000]자본(*)(연결)(천원)                               

In [55]:
KONEX.isin([0]).sum().sort_values(ascending=False)

주당순자산(원)                                                  1477
현재까지 감소한 주식의 총수(주)                                        1005
[U01E100099100]   당기순이익(순손실)(요약)(IFRS연결)(백만원)              337
[U01219300]CPA수정후 당기순이익(요약)(백만원)                           287
[U01B951020000]*주당순손익(요약)(IFRS연결)(원)                       190
[U01B951020000]*주당순손익(IFRS연결)(원)                           148
발행주식의 총수 (현재 발행한 주식수 - 현재 감소한 주식수)(주)                      139
[U01290400]*주당순손익(연결)(원)                                   129
현재까지 발행한 주식의 총수(주)                                          71
[U01D101000000]   당기순손익(요약)(IFRS)(백만원)                      66
[U01210000]매출액(요약)(백만원)                                     51
[U01210000]매출액(영업수익)(*)(천원)                                 48
[U01D101000000]   당기순손익(요약)(IFRS연결)(백만원)                    40
[U01311400]   당기순이익(요약)(백만원)                                19
[U01E100099100]   당기순이익(순손실)(요약)(IFRS)(백만원)                 16
[U01B100000000]매출액(수익)(요약)(IFRS)(백만원)                  

결측치 임시처리

In [56]:
KONEX.replace(0, np.nan, inplace=True)

In [57]:
KONEX.isna().sum().sort_values(ascending=False)

주당순자산(원)                                                  1477
종가(원)_x                                                   1477
[U01E100099100]   당기순이익(순손실)(요약)(IFRS연결)(백만원)             1477
[U01B951020000]*주당순손익(요약)(IFRS연결)(원)                      1330
[U01B951020000]*주당순손익(IFRS연결)(원)                          1288
[U01290400]*주당순손익(연결)(원)                                  1211
[U01D101000000]   당기순손익(요약)(IFRS연결)(백만원)                  1180
[U01B100000000]매출액(수익)(요약)(IFRS연결)(백만원)                   1142
[U01B100000000]매출액(수익)(*)(IFRS연결)(천원)                     1142
[U01D100000000]영업활동으로 인한 현금흐름(간접법)(*)(IFRS연결)(천원)         1140
[U01B840000000]당기순이익(손실)(요약)(IFRS연결)(백만원)                 1140
[U01C500011300]   당기순이익(총포괄손익)(요약)(IFRS연결)(백만원)           1140
[U01D100000000]영업활동으로 인한 현금흐름(간접법)(*)(요약)(IFRS연결)(백만원)    1140
[U01A600000000]자본(*)(IFRS연결)(백만원)                         1140
[U01A600000000]자본(*)(요약)(IFRS연결)(백만원)                     1140
[U01210000]매출액(영업수익)(*)(연결)(천원)                        

In [58]:
KONEX.drop(labels=['주당순자산(원)', '종가(원)_x', '[U01E100099100]   당기순이익(순손실)(요약)(IFRS연결)(백만원)','회사명_y'], axis=1, inplace=True)
KONEX.rename(columns={'회사명_x':'회사명', '종가(원)_y':'종가(원)'}, inplace=True)

In [59]:
KONEX.columns

Index(['회사명', '거래소코드', '회계년도', '현재까지 발행한 주식의 총수(주)', '현재까지 감소한 주식의 총수(주)',
       '발행주식의 총수 (현재 발행한 주식수 - 현재 감소한 주식수)(주)', '발행주식수(주)', '발행주식수(주).1',
       '[U01B840000000]당기순이익(손실)(요약)(IFRS연결)(백만원)',
       '[U01C500011300]   당기순이익(총포괄손익)(요약)(IFRS연결)(백만원)',
       '[U01D101000000]   당기순손익(요약)(IFRS연결)(백만원)',
       '[U01B840000000]당기순이익(손실)(요약)(IFRS)(백만원)',
       '[U01C500011300]   당기순이익(총포괄손익)(요약)(IFRS)(백만원)',
       '[U01D101000000]   당기순손익(요약)(IFRS)(백만원)',
       '[U01E100099100]   당기순이익(순손실)(요약)(IFRS)(백만원)',
       '[U01119300]      *당기순이익(요약)(백만원)', '[U01218800]당기순이익(요약)(백만원)',
       '[U01219300]CPA수정후 당기순이익(요약)(백만원)', '[U01311400]   당기순이익(요약)(백만원)',
       '[U01410100]   당기순이익(요약)(백만원)', '[U01550700]   당기순이익(요약)(백만원)',
       '[U01A600000000]자본(*)(요약)(IFRS연결)(백만원)',
       '[U01A600000000]자본(*)(IFRS연결)(백만원)',
       '[U01A600000000]자본(*)(요약)(IFRS)(백만원)', '[U01A600000000]자본(*)(IFRS)(천원)',
       '[U01119400]자본총계(요약)(천원)', '[U01160000]자본(*)(연결)(천원)',
       '[U01160000]자본(*)(천원)'

In [60]:
KONEX_FIRST = KONEX.drop_duplicates(subset=['회사명', '거래소코드', '회계년도'], keep='first', ignore_index=True)
KONEX_LAST = KONEX.drop_duplicates(subset=['회사명', '거래소코드', '회계년도'], keep='last', ignore_index=True)

In [61]:
KONEX_FIRST.isna().sum() - KONEX_LAST.isna().sum()

회사명                                                        0
거래소코드                                                      0
회계년도                                                       0
현재까지 발행한 주식의 총수(주)                                         4
현재까지 감소한 주식의 총수(주)                                        -5
발행주식의 총수 (현재 발행한 주식수 - 현재 감소한 주식수)(주)                     14
발행주식수(주)                                                   0
발행주식수(주).1                                                 0
[U01B840000000]당기순이익(손실)(요약)(IFRS연결)(백만원)                  0
[U01C500011300]   당기순이익(총포괄손익)(요약)(IFRS연결)(백만원)            0
[U01D101000000]   당기순손익(요약)(IFRS연결)(백만원)                   0
[U01B840000000]당기순이익(손실)(요약)(IFRS)(백만원)                    0
[U01C500011300]   당기순이익(총포괄손익)(요약)(IFRS)(백만원)              0
[U01D101000000]   당기순손익(요약)(IFRS)(백만원)                     0
[U01E100099100]   당기순이익(순손실)(요약)(IFRS)(백만원)                0
[U01119300]      *당기순이익(요약)(백만원)                           0
[U01218800]당기순이익(요약)(백만원

In [62]:
KONEX_FIRST.isna().sum()

회사명                                                         0
거래소코드                                                       0
회계년도                                                        0
현재까지 발행한 주식의 총수(주)                                         26
현재까지 감소한 주식의 총수(주)                                        377
발행주식의 총수 (현재 발행한 주식수 - 현재 감소한 주식수)(주)                      54
발행주식수(주)                                                    0
발행주식수(주).1                                                  0
[U01B840000000]당기순이익(손실)(요약)(IFRS연결)(백만원)                 425
[U01C500011300]   당기순이익(총포괄손익)(요약)(IFRS연결)(백만원)           425
[U01D101000000]   당기순손익(요약)(IFRS연결)(백만원)                  441
[U01B840000000]당기순이익(손실)(요약)(IFRS)(백만원)                   274
[U01C500011300]   당기순이익(총포괄손익)(요약)(IFRS)(백만원)             274
[U01D101000000]   당기순손익(요약)(IFRS)(백만원)                    300
[U01E100099100]   당기순이익(순손실)(요약)(IFRS)(백만원)               280
[U01119300]      *당기순이익(요약)(백만원)                            0
[U012188

In [63]:
KONEX_LAST.isna().sum()

회사명                                                         0
거래소코드                                                       0
회계년도                                                        0
현재까지 발행한 주식의 총수(주)                                         22
현재까지 감소한 주식의 총수(주)                                        382
발행주식의 총수 (현재 발행한 주식수 - 현재 감소한 주식수)(주)                      40
발행주식수(주)                                                    0
발행주식수(주).1                                                  0
[U01B840000000]당기순이익(손실)(요약)(IFRS연결)(백만원)                 425
[U01C500011300]   당기순이익(총포괄손익)(요약)(IFRS연결)(백만원)           425
[U01D101000000]   당기순손익(요약)(IFRS연결)(백만원)                  441
[U01B840000000]당기순이익(손실)(요약)(IFRS)(백만원)                   274
[U01C500011300]   당기순이익(총포괄손익)(요약)(IFRS)(백만원)             274
[U01D101000000]   당기순손익(요약)(IFRS)(백만원)                    300
[U01E100099100]   당기순이익(순손실)(요약)(IFRS)(백만원)               280
[U01119300]      *당기순이익(요약)(백만원)                            0
[U012188

In [64]:
KONEX_FIRST.fillna(KONEX_LAST, inplace=True)
KONEX_LAST.fillna(KONEX_LAST, inplace=True)
KONEX_FIRST.fillna(KONEX_LAST, inplace=True)

In [65]:
KONEX_FIRST.shape

(554, 52)

In [66]:
KONEX_market = KONEX_FIRST

In [67]:
KONEX_market.isna().sum()

회사명                                                         0
거래소코드                                                       0
회계년도                                                        0
현재까지 발행한 주식의 총수(주)                                          0
현재까지 감소한 주식의 총수(주)                                        333
발행주식의 총수 (현재 발행한 주식수 - 현재 감소한 주식수)(주)                       0
발행주식수(주)                                                    0
발행주식수(주).1                                                  0
[U01B840000000]당기순이익(손실)(요약)(IFRS연결)(백만원)                 425
[U01C500011300]   당기순이익(총포괄손익)(요약)(IFRS연결)(백만원)           425
[U01D101000000]   당기순손익(요약)(IFRS연결)(백만원)                  441
[U01B840000000]당기순이익(손실)(요약)(IFRS)(백만원)                   274
[U01C500011300]   당기순이익(총포괄손익)(요약)(IFRS)(백만원)             274
[U01D101000000]   당기순손익(요약)(IFRS)(백만원)                    300
[U01E100099100]   당기순이익(순손실)(요약)(IFRS)(백만원)               280
[U01119300]      *당기순이익(요약)(백만원)                            0
[U012188

발행주식수

In [68]:
발행주식수 = KONEX_market['발행주식수(주)'].fillna(KONEX_market['발행주식수(주).1'])
발행주식수 = 발행주식수.fillna(KONEX_market['발행주식의 총수 (현재 발행한 주식수 - 현재 감소한 주식수)(주)'])
발행주식수 = 발행주식수.fillna(KONEX_market['현재까지 발행한 주식의 총수(주)'] - KONEX_market['현재까지 감소한 주식의 총수(주)'])
KONEX_market['발행주식수'] = 발행주식수
KONEX_market.drop(labels=['발행주식수(주)', '발행주식수(주).1', '발행주식의 총수 (현재 발행한 주식수 - 현재 감소한 주식수)(주)', '현재까지 감소한 주식의 총수(주)', '현재까지 발행한 주식의 총수(주)'], axis=1, inplace=True)
KONEX_market['발행주식수'].isna().value_counts()

발행주식수
False    554
Name: count, dtype: int64

당기순이익

In [69]:
당기순이익 = (KONEX_market['당기순이익(천원)'] / 1000).fillna(KONEX_market['[U01B840000000]당기순이익(손실)(요약)(IFRS연결)(백만원)'])
당기순이익 = 당기순이익.fillna(KONEX_market['[U01C500011300]   당기순이익(총포괄손익)(요약)(IFRS연결)(백만원)'])
당기순이익 = 당기순이익.fillna(KONEX_market['[U01D101000000]   당기순손익(요약)(IFRS연결)(백만원)'])
당기순이익 = 당기순이익.fillna(KONEX_market['[U01B840000000]당기순이익(손실)(요약)(IFRS)(백만원)'])
당기순이익 = 당기순이익.fillna(KONEX_market['[U01C500011300]   당기순이익(총포괄손익)(요약)(IFRS)(백만원)'])
당기순이익 = 당기순이익.fillna(KONEX_market['[U01D101000000]   당기순손익(요약)(IFRS)(백만원)'])
당기순이익 = 당기순이익.fillna(KONEX_market['[U01E100099100]   당기순이익(순손실)(요약)(IFRS)(백만원)'])
당기순이익 = 당기순이익.fillna(KONEX_market['[U01119300]      *당기순이익(요약)(백만원)'])
당기순이익 = 당기순이익.fillna(KONEX_market['[U01218800]당기순이익(요약)(백만원)'])
당기순이익 = 당기순이익.fillna(KONEX_market['[U01219300]CPA수정후 당기순이익(요약)(백만원)'])
당기순이익 = 당기순이익.fillna(KONEX_market['[U01311400]   당기순이익(요약)(백만원)'])
당기순이익 = 당기순이익.fillna(KONEX_market['[U01410100]   당기순이익(요약)(백만원)'])
당기순이익 = 당기순이익.fillna(KONEX_market['[U01550700]   당기순이익(요약)(백만원)'])
KONEX_market['당기순이익'] = 당기순이익
KONEX_market.drop(
    labels=[
        '[U01B840000000]당기순이익(손실)(요약)(IFRS연결)(백만원)',
        '[U01C500011300]   당기순이익(총포괄손익)(요약)(IFRS연결)(백만원)',
        '[U01D101000000]   당기순손익(요약)(IFRS연결)(백만원)',
        '[U01B840000000]당기순이익(손실)(요약)(IFRS)(백만원)',
        '[U01C500011300]   당기순이익(총포괄손익)(요약)(IFRS)(백만원)',
        '[U01D101000000]   당기순손익(요약)(IFRS)(백만원)',
        '[U01E100099100]   당기순이익(순손실)(요약)(IFRS)(백만원)',
        '[U01119300]      *당기순이익(요약)(백만원)',
        '[U01218800]당기순이익(요약)(백만원)',
        '[U01219300]CPA수정후 당기순이익(요약)(백만원)',
        '[U01311400]   당기순이익(요약)(백만원)',
        '[U01410100]   당기순이익(요약)(백만원)',
        '[U01550700]   당기순이익(요약)(백만원)',
        '당기순이익(천원)'
    ],
    axis=1,
    inplace=True
)
KONEX_market['당기순이익'].isna().value_counts()

당기순이익
False    554
Name: count, dtype: int64

순자산

In [70]:
순자산 = KONEX_market['[U01A600000000]자본(*)(요약)(IFRS연결)(백만원)'].fillna(KONEX_market['[U01A600000000]자본(*)(IFRS연결)(백만원)'])
순자산 = 순자산.fillna(KONEX_market['[U01A600000000]자본(*)(요약)(IFRS)(백만원)'])
순자산 = 순자산.fillna(KONEX_market['[U01A600000000]자본(*)(IFRS)(천원)'] / 1000)
순자산 = 순자산.fillna(KONEX_market['[U01119400]자본총계(요약)(천원)'] / 1000)
순자산 = 순자산.fillna(KONEX_market['[U01160000]자본(*)(연결)(천원)'] / 1000)
순자산 = 순자산.fillna(KONEX_market['[U01160000]자본(*)(천원)'] / 1000)
KONEX_market['순자산'] = 순자산
KONEX_market.drop(
    labels=[
        '[U01A600000000]자본(*)(요약)(IFRS연결)(백만원)',
       '[U01A600000000]자본(*)(IFRS연결)(백만원)',
       '[U01A600000000]자본(*)(요약)(IFRS)(백만원)', '[U01A600000000]자본(*)(IFRS)(천원)',
       '[U01119400]자본총계(요약)(천원)', '[U01160000]자본(*)(연결)(천원)',
       '[U01160000]자본(*)(천원)'
    ],
    axis=1,
    inplace=True
)
KONEX_market['순자산'].isna().value_counts()

순자산
False    554
Name: count, dtype: int64

영업현금흐름

In [71]:
영업현금흐름 = KONEX_market['[U01D100000000]영업활동으로 인한 현금흐름(간접법)(*)(요약)(IFRS연결)(백만원)'].fillna(KONEX_market['[U01D100000000]영업활동으로 인한 현금흐름(간접법)(*)(요약)(IFRS)(백만원)'])
영업현금흐름 = 영업현금흐름.fillna(KONEX_market['[U01410000]영업활동으로 인한 현금흐름(요약)(백만원)'])
영업현금흐름 = 영업현금흐름.fillna(KONEX_market['[U01D100000000]영업활동으로 인한 현금흐름(간접법)(*)(IFRS연결)(천원)'] / 1000)
영업현금흐름 = 영업현금흐름.fillna(KONEX_market['[U01D100000000]영업활동으로 인한 현금흐름(간접법)(*)(IFRS)(천원)'] / 1000)
영업현금흐름 = 영업현금흐름.fillna(KONEX_market['[U01410000]영업활동으로 인한 현금흐름(*)(연결)(천원)'] / 1000)
영업현금흐름 = 영업현금흐름.fillna(KONEX_market['[U01410000]영업활동으로 인한 현금흐름(*)(천원)'] / 1000)
KONEX_market['영업현금흐름'] = 영업현금흐름
KONEX_market.drop(
    labels=[
        '[U01D100000000]영업활동으로 인한 현금흐름(간접법)(*)(요약)(IFRS연결)(백만원)',
       '[U01D100000000]영업활동으로 인한 현금흐름(간접법)(*)(요약)(IFRS)(백만원)',
       '[U01410000]영업활동으로 인한 현금흐름(요약)(백만원)',
       '[U01D100000000]영업활동으로 인한 현금흐름(간접법)(*)(IFRS연결)(천원)',
       '[U01D100000000]영업활동으로 인한 현금흐름(간접법)(*)(IFRS)(천원)',
       '[U01410000]영업활동으로 인한 현금흐름(*)(연결)(천원)',
       '[U01410000]영업활동으로 인한 현금흐름(*)(천원)'
    ],
    axis=1,
    inplace=True
)
KONEX_market['영업현금흐름'].isna().value_counts()

영업현금흐름
False    554
Name: count, dtype: int64

매출액

In [72]:
매출액 = KONEX_market['[U01B100000000]매출액(수익)(요약)(IFRS연결)(백만원)'].fillna(KONEX_market[ '[U01B100000000]매출액(수익)(요약)(IFRS)(백만원)'])
매출액 = 매출액.fillna(KONEX_market['[U01210000]매출액(요약)(백만원)'])
매출액 = 매출액.fillna(KONEX_market['[U01B100000000]매출액(수익)(*)(IFRS연결)(천원)'] / 1000)
매출액 = 매출액.fillna(KONEX_market[ '[U01B100000000]매출액(수익)(*)(IFRS)(천원)'] / 1000)
매출액 = 매출액.fillna(KONEX_market['[U01210000]매출액(영업수익)(*)(연결)(천원)'] / 1000)
매출액 = 매출액.fillna(KONEX_market['[U01210000]매출액(영업수익)(*)(천원)'] / 1000)
KONEX_market['매출액'] = 매출액
KONEX_market.drop(
    labels=[
       '[U01B100000000]매출액(수익)(요약)(IFRS연결)(백만원)',
       '[U01B100000000]매출액(수익)(요약)(IFRS)(백만원)', '[U01210000]매출액(요약)(백만원)',
       '[U01B100000000]매출액(수익)(*)(IFRS연결)(천원)',
       '[U01B100000000]매출액(수익)(*)(IFRS)(천원)',
       '[U01210000]매출액(영업수익)(*)(연결)(천원)', '[U01210000]매출액(영업수익)(*)(천원)'
    ],
    axis=1,
    inplace=True
)
KONEX_market['매출액'].isna().value_counts()

매출액
False    539
True      15
Name: count, dtype: int64

주당순손익

In [73]:
주당순손익 = KONEX_market['주당순이익(원)'].fillna(KONEX_market['[U01B951020000]*주당순손익(요약)(IFRS연결)(원)'])
주당순손익 = 주당순손익.fillna(KONEX_market['[U01B951020000]*주당순손익(요약)(IFRS)(원)'])
주당순손익 = 주당순손익.fillna(KONEX_market['[U01219000]주당순이익(요약)(원)'])
주당순손익 = 주당순손익.fillna(KONEX_market['[U01B951020000]*주당순손익(IFRS연결)(원)'])
주당순손익 = 주당순손익.fillna(KONEX_market['[U01B951020000]*주당순손익(IFRS)(원)'])
주당순손익 = 주당순손익.fillna(KONEX_market['[U01290400]*주당순손익(연결)(원)'])
주당순손익 = 주당순손익.fillna(KONEX_market['[U01290400]*주당순손익(원)'])
KONEX_market['주당순손익'] = 주당순손익
KONEX_market.drop(
    labels=[
       '[U01B951020000]*주당순손익(요약)(IFRS연결)(원)',
       '[U01B951020000]*주당순손익(요약)(IFRS)(원)', '[U01219000]주당순이익(요약)(원)',
       '[U01B951020000]*주당순손익(IFRS연결)(원)', '[U01B951020000]*주당순손익(IFRS)(원)',
       '[U01290400]*주당순손익(연결)(원)', '[U01290400]*주당순손익(원)', '주당순이익(원)'
    ],
    axis=1,
    inplace=True
)
KONEX_market['주당순손익'].isna().value_counts()

주당순손익
False    554
Name: count, dtype: int64

In [74]:
KONEX_market.columns

Index(['회사명', '거래소코드', '회계년도', '종가(원)', '발행주식수', '당기순이익', '순자산', '영업현금흐름',
       '매출액', '주당순손익'],
      dtype='object')

PER

In [75]:
주당순이익2 = KONEX_market['당기순이익'] / KONEX_market['발행주식수']
KONEX_market['주당순손익'] = KONEX_market['주당순손익'].fillna(주당순이익2)
KONEX_market['PER'] = KONEX_market['종가(원)'] / KONEX_market['주당순손익']

In [76]:
KONEX_market['PER'].value_counts().sort_index(ascending=True)

PER
-676.923077     1
-596.153846     1
-511.111111     1
-383.333333     1
-215.000000     1
               ..
 393.888889     1
 1198.750000    1
 1700.000000    1
 2860.000000    1
 5050.000000    1
Name: count, Length: 554, dtype: int64

PBR

In [77]:
주당순자산2 = KONEX_market['순자산'] / KONEX_market['발행주식수']
KONEX_market['주당순자산'] = 주당순자산2
KONEX_market['PBR'] = KONEX_market['종가(원)'] / KONEX_market['주당순자산']

In [78]:
KONEX_market['PBR'].value_counts().sort_index(ascending=True)

PBR
-7.881139e+11    1
-6.714752e+10    1
-6.451458e+10    1
-4.086287e+10    1
-3.292238e+10    1
                ..
 1.040800e+11    1
 1.564928e+11    1
 2.777240e+11    1
 4.687893e+11    1
 1.284197e+12    1
Name: count, Length: 554, dtype: int64

PCR

In [79]:
KONEX_market['PCR'] = KONEX_market['종가(원)'] * KONEX_market['발행주식수'] / KONEX_market['영업현금흐름']

In [80]:
KONEX_market['PCR'].value_counts().sort_index(ascending=True)

PCR
-8.179594e+09    1
-4.614400e+09    1
-1.366154e+09    1
-7.998372e+08    1
-6.930630e+08    1
                ..
 5.541067e+08    1
 6.253499e+08    1
 7.458368e+08    1
 1.052488e+09    1
 2.369203e+09    1
Name: count, Length: 554, dtype: int64

PSR

In [81]:
KONEX_market['PSR'] = KONEX_market['종가(원)'] * KONEX_market['발행주식수'] / KONEX_market['매출액']

In [82]:
KONEX_market['PSR'].value_counts().sort_index(ascending=True)

PSR
1.036762e+05    1
1.068005e+05    1
1.299647e+05    1
1.335505e+05    1
1.490294e+05    1
               ..
3.890952e+09    1
4.612001e+09    1
4.929577e+09    1
5.445335e+09    1
2.430257e+10    1
Name: count, Length: 539, dtype: int64

KONEX_market_indicators

In [83]:
KONEX_market_indicators = KONEX_market[['회사명', '거래소코드', '회계년도', 'PER', 'PBR', 'PCR', 'PSR']]

In [84]:
KONEX_market_indicators.isna().sum()

회사명       0
거래소코드     0
회계년도      0
PER       0
PBR       0
PCR       0
PSR      15
dtype: int64

# to_csv

In [85]:
market_indicators = pd.concat([KOSPI_KOSDAQ_market_indicators, KONEX_market_indicators], ignore_index=True)

In [86]:
market_indicators.to_csv("./datasets/TS2000/상장_시장지표.csv")